In [ ]:
# Utility imports
from __future__ import print_function
import base64
import json
import warnings
import sys
import numpy as np
from StringIO import StringIO

# Streaming imports
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

# Object detection imports
sys.path.append("./object_detection") # This is needed since the notebook is stored in the object_detection folder.
import tensorflow as tf
from PIL import Image
from object_detection.utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util

In [ ]:
# Model Name
MODEL_FILE = 'ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb' 

# List of the strings that is used to add correct label for each box.
COCO_PATH_TO_LABELS = os.path.join('object_detection', 'data', 'mscoco_label_map.pbtxt')
COCO_NUM_CLASSES = 90

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 15)

In [ ]:
# Load Labels & Categories
label_map = label_map_util.load_labelmap(COCO_PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=COCO_NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)    

# Load Frozen Network Model
with tf.gfile.FastGFile(MODEL_FILE, 'rb') as f:
    model_data = f.read()

# Broadcast model in Spark
model_data_bc = sc.broadcast(model_data)

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
def apply_detect(payload):
    event = json.loads(payload[1])
    
    v_sectors = ['top', 'middle', 'bottom']
    h_sectors = ['left', 'center', 'right']
    
    # Loads the image data from the URL:
    decoded = base64.b64decode(event['image']) 
    stream = StringIO(decoded)
        
    image = Image.open(stream)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    print('test')

    with tf.Graph().as_default() as g:
        # Load Network Graph
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(model_data_bc.value)
        tf.import_graph_def(graph_def, name='')
        
        # Runs a tensor flow session that loads the
        with tf.Session() as sess:
            # Get handles to input and output tensors
            print('sess')            
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
              ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]

            
    # Transforming for output
    result= {'timestamp': event['timestamp'], 
             'camera_id': event['camera_id'],
             'objects': []
            }
    
    for i in range(len(output_dict['detection_classes'])):
        # Get score and round on 2 digits for nice output
        score = round(output_dict['detection_scores'][i], 2)
        
        if score > 0.6:  # Only keep objects over treshold
            
            # Get label for category id
            cat_id = output_dict['detection_classes'][i]
            label = category_index[cat_id]['name']
            
            # Get position of object box as [ymin, xmin, ymax, xmax]
            box = output_dict['detection_boxes'][i]
            
            # Transform into sector of image
            width = box[3] - box[1]
            h_pos = box[1] + width / 2.0
            height = box[2] - box[0]
            v_pos = box[0] + height / 2.0
            h_sector = min(int(h_pos * 3), 2)  # 0: left, 1: center, 2: right
            v_sector = min(int(v_pos * 3), 2)  # 0: top, 1: middle, 2: bottom
            
            # Add object information to result
            result['objects'].append({
                           'label': label, 
                           'score': score, 
                           'sector': (v_sectors[v_sector], h_sectors[h_sector])
                          })

    return result

In [ ]:
# Stream Settings
zkQuorum = 'localhost:2181'
topic = 'pycturestream'
ssc = StreamingContext(sc, 3)  # Update every 3 seconds
kafka_stream = KafkaUtils.createStream(ssc, 
                              zkQuorum, 
                              "spark-streaming-consumer", 
                              {topic: 1})

# Transform stream payload
parsed_dstream = kafka_stream.map(lambda x: apply_detect(x))
parsed_dstream.pprint()

# Actually start Streaming Context
ssc.start()  
ssc.awaitTermination()